### Un perceptrón básico en pytorch

Ecribamos la codificación de la red neuronal más simple posible:

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

Definimos una red con una sola neurona.

In [ ]:
class Red(nn.Module):
  def __init__(self):
    super(Red,self).__init__()
    self.fc1 = nn.Linear(1,1)
  
  def forward(self, x):
    x = self.fc1(x)
    return x

In [ ]:
red = Red()
print(red)

Red(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)


Entonces es posible echar un vistazo a los parámetros de la red. Los parámetros son optimizados automáticamente por la red. Los hiperparámetros, como la velocidad de aprendizaje, requieren el ajuste por parte de un humano (hasta ahora, al menos).

In [ ]:
print(list(red.parameters()))

[Parameter containing:
tensor([[-0.5807]], requires_grad=True), Parameter containing:
tensor([-0.9690], requires_grad=True)]


Trabajemos con esta red neuronal básica.

In [ ]:
entrada = Variable(torch.randn(1,1,1), requires_grad=True)
print(entrada)

tensor([[[1.9279]]], requires_grad=True)


Sí, acabamos de crear un número aleatorio con PyTorch. Es un tensor con una sola dimensión (alternativamente, `torch.FloatTensor([[[1]]])` crearía un tensor equivalente con el valor `1`). **Establecer `require_grad` significa que es una variable optimizable. Entonces puedes lanzar este número a través de la red no aprendida**:

In [ ]:
salida = red(entrada)
print(salida)

tensor([[[-2.0886]]], grad_fn=<AddBackward0>)


### Función de pérdida y Optimizador

In [ ]:
import torch.optim as optim
def criterio(salida, etiqueta):
  return (etiqueta -salida)** 2

optimizador = optim.SGD(red.parameters(), lr=0.01, momentum = 0.5)

Definamos ahora un conjunto de datos de entrenamiento. Para este caso  solo vamos a enseñar a la red cómo triplicar un número: nuestro objetivo para el  perceptrón  Ax + b será que A = 3 y b = 0. Un conjunto de datos de entrenamiento simple es:

In [ ]:
datos = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]

Entonces, el bucle de entrenamiento se ve como:

In [ ]:
for epoch in range(100):
  for i, datos1 in enumerate(datos):
    X,Y =iter(datos1)
    X,Y = Variable(torch.FloatTensor([X]), requires_grad =True), Variable(torch.FloatTensor([Y]),
                                                                        requires_grad=False)
    
    optimizador.zero_grad()
    salidas = red(X)
    f_perdida =criterio(salidas, Y)
    f_perdida.backward()
    optimizador.step()
    if (i% 10 == 0):
      print("Epoca {}-FuncionPerdida: {}".format(epoch, f_perdida.data[0]))
    

Epoca 0-FuncionPerdida: 20.6999454498291
Epoca 1-FuncionPerdida: 0.9244413375854492
Epoca 2-FuncionPerdida: 0.006411971524357796
Epoca 3-FuncionPerdida: 0.056640781462192535
Epoca 4-FuncionPerdida: 0.013704035431146622
Epoca 5-FuncionPerdida: 0.022266890853643417
Epoca 6-FuncionPerdida: 0.01642902009189129
Epoca 7-FuncionPerdida: 0.01595809869468212
Epoca 8-FuncionPerdida: 0.013971650041639805
Epoca 9-FuncionPerdida: 0.012708649970591068
Epoca 10-FuncionPerdida: 0.011396888643503189
Epoca 11-FuncionPerdida: 0.010274446569383144
Epoca 12-FuncionPerdida: 0.00924444105476141
Epoca 13-FuncionPerdida: 0.008323724381625652
Epoca 14-FuncionPerdida: 0.0074927592650055885
Epoca 15-FuncionPerdida: 0.006745329592376947
Epoca 16-FuncionPerdida: 0.006072300486266613
Epoca 17-FuncionPerdida: 0.005466494709253311
Epoca 18-FuncionPerdida: 0.004921008367091417
Epoca 19-FuncionPerdida: 0.004430110566318035
Epoca 20-FuncionPerdida: 0.003988094162195921
Epoca 21-FuncionPerdida: 0.0035902149975299835
Epoca

¿Llegamos a Ax + b (3x + 0)? Casi:

In [ ]:
print(list(red.parameters()))

[Parameter containing:
tensor([[2.9998]], requires_grad=True), Parameter containing:
tensor([0.0011], requires_grad=True)]


¿Qué hay acerca de las predicciones?

In [ ]:
print(red(Variable(torch.Tensor([[[1]]]))))

tensor([[[3.0009]]], grad_fn=<AddBackward0>)


### Multilayer Perceptron

El mismo código  todavía funciona para una red de dos capas (o más que eso). Solo cambia la forma en que se construye la red. Tenga en cuenta que las capas deben coincidir en términos de cantidad de salidas de una capa y de entradas a la siguiente:

In [ ]:
class Red(nn.Module):
  def __init__(self):
    super(Red, self).__init__()
    self.fc1 = nn.Linear(1, 10)
    self.fc2 = nn.Linear(10,1)
  
  def forward(self, x):
    x = self.fc2(self.fc1(x))
    return x

 ### GPU

Es notablemente fácil con PyTorch pasar la computación a la GPU, asumiendo que puede permitirse uno en estos tiempos de escasez de DDR y minería criptográfica. Simplemente desplace la red y las variables a la GPU con `cuda()`:

In [ ]:
red = Red()
red.cuda()

Red(
  (fc1): Linear(in_features=1, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)

In [ ]:
print(list(red.parameters()))

[Parameter containing:
tensor([[ 0.0852],
        [ 0.8926],
        [-0.4824],
        [ 0.8563],
        [ 0.2060],
        [-0.8734],
        [-0.5102],
        [ 0.5776],
        [ 0.5295],
        [ 0.3026]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.6319,  0.5182,  0.0059,  0.4588, -0.4930, -0.0521, -0.2336,  0.6435,
         0.3214, -0.3157], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[-0.1780, -0.1619,  0.1026,  0.2500,  0.1539,  0.2664,  0.1547, -0.1572,
         -0.0159,  0.2245]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1121], device='cuda:0', requires_grad=True)]


In [ ]:
entrada =Variable(torch.randn(1,1,1)).cuda()
print (entrada)

tensor([[[3.1484]]], device='cuda:0')


Las redes neuronales solo funcionan porque cada neurona tiene cierta no linealidad. Lo que sorprende a mi mente como alguien que creció con sigmoide y tanh, es que el mejor tipo de no linealidad en estos días es ReLU, o Unidad Lineal Rectificada. Es decir, si la suma de la neurona es negativa, establézcase en cero, de lo contrario proceda como de costumbre.

Con gusto puedes reconstruir la red con ReLU y se entrenará de la misma manera.

In [ ]:
red.zero_grad()

In [ ]:
salida = red(entrada)
print(salida)

tensor([[[-1.2362]]], device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:
import torch.optim as optim
optimizador = optim.SGD(red.parameters(), lr = 0.001, momentum=0.4)
criterio = nn.MSELoss()

In [ ]:
datos =[(1,3), (2, 6), (3, 9), (4, 12), (5, 15), (6, 18), (7, 21), (8, 24), (9, 27), (10, 30)]

El bucle de entrenamiento para este caso es:

In [ ]:
for epoch in range(100):
  for i, datos1 in enumerate(datos):
    X, Y =iter(datos1)
    X, Y = Variable(torch.FloatTensor([X]), requires_grad=True).cuda(), Variable(torch.FloatTensor([Y]), requires_grad=False).cuda()
      
    optimizador.zero_grad()
    y_pred = red(X)
    salida = criterio(y_pred,Y)
    salida.backward()
    optimizador.step()
  
  if (epoch %20 == 0.0):
      print("Epoca {} - Funcion_perdida: {}".format(epoch, salida))



Epoca 0 - Funcion_perdida: 0.09425553679466248
Epoca 20 - Funcion_perdida: 0.0016533053712919354
Epoca 40 - Funcion_perdida: 0.0005907497252337635
Epoca 60 - Funcion_perdida: 0.00021156984439585358
Epoca 80 - Funcion_perdida: 7.597882358822972e-05


In [ ]:
print(list(red.parameters()))

[Parameter containing:
tensor([[-0.0085],
        [ 0.9520],
        [-0.4881],
        [ 1.2326],
        [ 0.3403],
        [-0.8229],
        [-0.4917],
        [ 0.5901],
        [ 0.6290],
        [ 0.5193]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([ 0.6375,  0.3782,  0.0785,  0.2714, -0.5480,  0.0689, -0.1620,  0.5586,
         0.2278, -0.3978], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[-0.1934,  0.4902, -0.2707,  0.9529,  0.3899, -0.3894, -0.2118,  0.2298,
          0.3845,  0.5371]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0219], device='cuda:0', requires_grad=True)]
